In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter
import csv
import pandas as pd

In [50]:
ulasan=pd.read_csv('../dataset/Hasil.csv')
ulasan.drop(ulasan.tail(3).index, inplace=True)
sentiment_count = ulasan['polarity'].value_counts()
sentiment_count


polarity
negative    608
positive    392
Name: count, dtype: int64

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

vectorizer = TfidfVectorizer(max_features=2000, max_df=0.75, min_df=5)  # Tambahkan max_df dan min_df

Encoder = LabelEncoder()
ulasan['polarity']  = Encoder.fit_transform(ulasan['polarity'] )

x_res =vectorizer.fit_transform(ulasan['content_preprocessing'])
y_res =ulasan['polarity']
X_train, X_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2,random_state=42)


print(f'Dataset sebelum SMOTE : {Counter(y_train)}')
#penerapan smote
smote = SMOTE(sampling_strategy='auto')
X_smote, Y_smote = smote.fit_resample(X_train, y_train)
print(f'Dataset sesudah SMOTE : {Counter(Y_smote)}')



Dataset sebelum SMOTE : Counter({0: 484, 1: 316})
Dataset sesudah SMOTE : Counter({0: 484, 1: 484})


In [52]:
print(sorted(Counter(Y_smote).items()))


[(0, 484), (1, 484)]


In [54]:
df = pd.DataFrame(X_smote)
df.rename(columns={0:'TF-IDF'}, inplace=True)
df['sentimen'] = Y_smote
# mengembalikan kalimat asli dari tfidf
feature_names = vectorizer.get_feature_names_out()

kalimat_asli = []
for index, row in df.iterrows():
    vektor_ulasan = X_smote[index]
    kata_kunci = [feature_names[i] for i in vektor_ulasan.indices]
    kalimat_asli.append(' '.join(kata_kunci))

# tambahkan kolom baru dengan kalimat asli ke dalam data frame
df['kalimat_asli'] = kalimat_asli
df.to_csv('../smote/data_smote.csv', index=False)
#mengambil data sintetik
df_sintetik = df.iloc[800:]
#menyimpan dalam bentuk csv
df_sintetik.to_csv('../smote/data_sintetik.csv', index=False)
df


,TF-IDF,sentimen,kalimat_asli
0,"(0, 473)\t0.20882954524218905\n (0, 796)\t0...",0,mungkin viral kata kemarin nyobain ph sih bau ...
1,"(0, 717)\t0.16014122263944605\n (0, 8)\t0.2...",0,stop ajaa purging gacocok sii dalem bahan taku...
2,"(0, 70)\t0.27139091758774325\n (0, 717)\t0....",0,batu stop tempat jalan minggu timbul putus mak...
3,"(0, 711)\t0.25408003333643114\n (0, 553)\t0...",0,sold paket wa preloved gatel minggu berat rasa...
4,"(0, 289)\t0.18794128659198747\n (0, 177)\t0...",0,ig di ering kuat salicylic pori jauh hijau ben...
...,...,...,...
963,"(0, 374)\t0.0220024394093043\n (0, 550)\t0....",1,kompres pakai hari deh cocok kalau suka nya ke...
964,"(0, 769)\t0.10547838397022413\n (0, 560)\t0...",1,toner pas npure breakout bikin jadi beli kandu...
965,"(0, 307)\t0.06312386358990682\n (0, 138)\t0...",1,jadi centella wajah kali buat harga tekstur wa...
966,"(0, 318)\t0.19355708559704443\n (0, 446)\t0...",1,jerawat memang muka ku buat tenang keluar teta...


In [57]:
test = pd.DataFrame(X_test)
test.rename(columns={0:'TF-IDF'}, inplace=True)
test['sentimen'] = y_test.reset_index(drop=True)
# mengembalikan kalimat asli dari tfidf
feature_names = vectorizer.get_feature_names_out()

kalimat_asli = []
for index, row in test.iterrows():
    vektor_ulasan = X_test[index]
    kata_kunci = [feature_names[i] for i in vektor_ulasan.indices]
    kalimat_asli.append(' '.join(kata_kunci))

# tambahkan kolom baru dengan kalimat asli ke dalam data frame
test['kalimat_asli'] = kalimat_asli
test

full_Smote = pd.concat([df, test], axis=0)

full_Smote.to_csv('../smote/data_smote_full.csv', index=False)


label_test=full_Smote['sentimen'].value_counts()


